In [16]:
""" A test script for monitoring vegetation change over a predefined area of
    interest by means of Sentinel-2 data from the Norwegian National Ground
    Segment (NBS) read through OPeNDAP.

    Made as a show case using Python 2.7

===========================================================
Name:          monitor_change_sentinel_2
Author(s):     Trygve Halsne 19.09.2018 (dd.mm.YYYY)
Modifications:
Copyright:     (c) Norwegian Meteorological Institute, 2018
===========================================================
"""


from netCDF4 import Dataset
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
from skimage import exposure

In [17]:
def linearStretch(image, pLow, pHigh):
    """Operator for linear stretching of histogram."""

    tmp_image = []
    for i in range(image.shape[np.argmin(image.shape)]):
        band = image[:,:,i]
        iMin, iMax = np.percentile(band[~np.isnan(band)], (pLow, 100-pHigh))
        band_rescaled = exposure.rescale_intensity(band, in_range=(iMin, iMax))
        tmp_image.append(band_rescaled)
    img_rescale = np.dstack(tmp_image)

    return img_rescale

def get_uint8_image( scaled_image, vmin, vmax):
    ''' Scale image from float (or any) input array to uint8
    Parameters
    ----------
        image : 2D matrix
        vmin : float - minimum value
        vmax : float - maximum value
    Returns
    -------
        2D matrix
    '''
    # redistribute into range [0,255]
    uint8Image = 255 * ((scaled_image - vmin) / float((vmax - vmin)))
    uint8Image[uint8Image < 0] = 0
    uint8Image[uint8Image > 255] = 255
    uint8Image[~np.isfinite(uint8Image)] = 0

    return uint8Image.astype('uint8')

def save_plot(output_name, data):
    ''' Stores plot on local machine '''
    fig = plt.figure(frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(data)
    ax.set_aspect('auto')
    ax.text(0.025, 0.95, date,
        verticalalignment='center', fontsize=20,
        transform=ax.transAxes, color='white',
        bbox={'facecolor':'black','alpha':0.5, 'edgecolor':'gray', 'boxstyle':'round'})
    fig.savefig(output_name)
    plt.close('all')


# OPeNDAP urls for reading data
urls = ['http://nbstds.met.no/thredds/fileServer/NBS/S2A/2015/08/22/S2A_OPER_PRD_MSIL1C_PDMC_20160706T181324_R008_V20150822T104035_20150822T104035/S2A_OPER_MSI_L1C_TL_EPA__20160706T123006_A000863_T32VPM_N02_04.nc',
        'http://nbstds.met.no/thredds/fileServer/NBS/S2A/2015/08/15/S2A_OPER_PRD_MSIL1C_PDMC_20160705T144923_R051_V20150815T105041_20150815T105041/S2A_OPER_MSI_L1C_TL_EPA__20160705T124729_A000763_T32VPM_N02_04.nc']
files = ['./S2A_OPER_MSI_L1C_TL_EPA__20160706T123006_A000863_T32VPM_N02_04.nc',
         './S2A_OPER_MSI_L1C_TL_EPA__20160705T124729_A000763_T32VPM_N02_04.nc']

# for i, file in enumerate(files):
#     i=0
#     url=urls[i]
#     t0 = dt.datetime.now()

#     # Reading netCDF files with Python
#     # http://www.ceda.ac.uk/static/media/uploads/ncas-reading-2015/10_read_netcdf_python.pdf

#     # open a netCDF file from python
#     # http://www.ceda.ac.uk/static/media/uploads/ncas-reading-2015/10_read_netcdf_python.pdf
#     ncin = Dataset(file)

#     x0 = 1100; x1  = 2100; y0 = 2000; y1 = 3000 # Area covering Gardermoen

#     b3_data = ncin.variables['B3'][0,y0:y1,x0:x1]
#     b4_data = ncin.variables['B4'][0,y0:y1,x0:x1]
#     b8_data = ncin.variables['B8'][0,y0:y1,x0:x1]
#     print("Read all variables in ", dt.datetime.now() - t0)

#     datetime = ncin.getncattr('start_time')
#     date = datetime.split('T')[0]


#     # Normalized Difference Vegetation Index (NDVI) uses the NIR and red channels to measure healthy vegetation.
#     # If you want to calculate it manually, this is the formula you can use.
#     # NDVI always generates a value between -1 and +1.
#     # It's really just a standardized way to measure healthy vegetation.
#     rgb_false_color_veg = linearStretch(np.dstack([b8_data,b4_data,b3_data]),0.1,0.1)
#     NDVI = 0.0 + (b8_data.astype(np.float32) - b4_data.astype(np.float32))/(b8_data.astype(np.float32) + b4_data.astype(np.float32))*1.0

#     imagery = {'fcv': rgb_false_color_veg,
#                'ndvi': NDVI}

#     # Saving plots to file
#     for composite,img in imagery.iteritems():
#         output_name = str(composite + '_' + str(i) + '.png')
#         save_plot(output_name, img)
#     ncin.close()

Read all variables in  0:00:00.295965


AttributeError: 'dict' object has no attribute 'iteritems'

In [19]:
i=0
t0 = dt.datetime.now()

# Reading netCDF files with Python
# http://www.ceda.ac.uk/static/media/uploads/ncas-reading-2015/10_read_netcdf_python.pdf

# open a netCDF file from python
# http://www.ceda.ac.uk/static/media/uploads/ncas-reading-2015/10_read_netcdf_python.pdf

ncin = Dataset(files[0])
ncin.variables.keys()

odict_keys(['time', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B8A', 'lat', 'lon', 'x', 'y', 'view_zenith_B1', 'view_azimuth_B1', 'view_zenith_B2', 'view_azimuth_B2', 'view_zenith_B3', 'view_azimuth_B3', 'view_zenith_B4', 'view_azimuth_B4', 'view_zenith_B5', 'view_azimuth_B5', 'view_zenith_B6', 'view_azimuth_B6', 'view_zenith_B7', 'view_azimuth_B7', 'view_zenith_B8', 'view_azimuth_B8', 'view_zenith_B8A', 'view_azimuth_B8A', 'view_zenith_B9', 'view_azimuth_B9', 'view_zenith_B10', 'view_azimuth_B10', 'view_zenith_B11', 'view_azimuth_B11', 'view_zenith_B12', 'view_azimuth_B12', 'sun_zenith', 'sun_azimuth', 'view_zenith_mean', 'view_azimuth_mean', 'cirrus_clouds', 'opaque_clouds', 'projection', 'Level_1C_User_Product_metadata', 'Level_1C_DataStrip_ID_metadata', 'Granules_Level_1C_Tile_ID_metadata'])

In [20]:
x0 = 1100; x1  = 2100; y0 = 2000; y1 = 3000 # Area covering Gardermoen

b3_data = ncin.variables['B3'][0,y0:y1,x0:x1]
b4_data = ncin.variables['B4'][0,y0:y1,x0:x1]
b8_data = ncin.variables['B8'][0,y0:y1,x0:x1]
print("Read all variables in ", dt.datetime.now() - t0)

datetime = ncin.getncattr('start_time')
datetime

Read all variables in  0:04:52.676396


'2015-08-22T10:40:35.460Z'

In [ ]:
date = datetime.split('T')[0]


# Normalized Difference Vegetation Index (NDVI) uses the NIR and red channels to measure healthy vegetation.
# If you want to calculate it manually, this is the formula you can use.
# NDVI always generates a value between -1 and +1.
# It's really just a standardized way to measure healthy vegetation.
rgb_false_color_veg = linearStretch(np.dstack([b8_data,b4_data,b3_data]),0.1,0.1)
NDVI = 0.0 + (b8_data.astype(np.float32) - b4_data.astype(np.float32))/(b8_data.astype(np.float32) + b4_data.astype(np.float32))*1.0

imagery = {'fcv': rgb_false_color_veg,
           'ndvi': NDVI}

In [23]:
imagery.keys()

dict_keys(['fcv', 'ndvi'])

In [24]:
imagery

{'fcv': masked_array(
   data=[[[0.53824092, 0.32897181, 0.30537883],
          [0.55200765, 0.3070092 , 0.2967033 ],
          [0.55449331, 0.28738304, 0.27356854],
          ...,
          [0.44034417, 0.35794376, 0.30364372],
          [0.44397706, 0.34392507, 0.29496819],
          [0.48527725, 0.27429894, 0.2388664 ]],
 
         [[0.4751434 , 0.29252323, 0.27183343],
          [0.58776291, 0.33177555, 0.30075188],
          [0.56481836, 0.33271012, 0.30769231],
          ...,
          [0.42963671, 0.39813066, 0.30537883],
          [0.41300191, 0.3649531 , 0.29323308],
          [0.47246654, 0.33504657, 0.30306536]],
 
         [[0.33021033, 0.15794385, 0.14748409],
          [0.46749522, 0.27756996, 0.24407172],
          [0.44053537, 0.26775688, 0.2388664 ],
          ...,
          [0.42695985, 0.39719608, 0.3227299 ],
          [0.42772467, 0.41542037, 0.3227299 ],
          [0.46520076, 0.3962615 , 0.31983806]],
 
         ...,
 
         [[0.28221797, 0.02990653, 0.0532099

In [26]:
# Saving plots to file
for composite,img in imagery.items():
    output_name = str(composite + '_' + str(i) + '.png')
    save_plot(output_name, img)
ncin.close()